# Applied Data Science Capstone - Week 3

## Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

#### Author: Alejandro Del Campo

##### 
##### First of all, instal all the libraries that I will need in this step

In [1]:
import pandas as pd
!pip install lxml bs4 html5lib
import lxml

##### 
##### Next, copy the url and use the read_html command to read all de tables of the URL, then introduce the first table in the dataframe

In [2]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitable=pd.read_html(page,flavor='html5lib')
toronto_df=wikitable[0]
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### The shape of the original dataframe

In [3]:
print('The  shape of dataframe is:',toronto_df.shape)

The  shape of dataframe is: (180, 3)


##### Let's drop the Postal Codes without any borough assigned

In [4]:
#Let's drop the row without Borough and Neighbourhood
df_to=toronto_df[toronto_df.Borough != 'Not assigned']
print('The new shape of dataframe is:',df_to.shape)
df_to = df_to.sort_values(by=['Postal Code','Borough'])
df_to=df_to.reset_index(drop=True)
df_to.head()

The new shape of dataframe is: (103, 3)


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df_to.shape

(103, 3)

### Step 2: Obtaining the coordinates of Toronto Neighbourhoods

In [6]:
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data
coord = pd.read_csv("Geospatial_data.csv", delimiter=",")

--2021-02-11 10:08:06--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.176, 169.63.96.194
Connecting to cocl.us (cocl.us)|169.63.96.176|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2021-02-11 10:08:06--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-02-11 10:08:06--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-02-11 10:08:07--  https://ibm.box.com/public/static/9afzr83p

In [7]:
coord[0:5]

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
coord.shape

(103, 3)

#### I have in the dataframe "coord" the coordinates of all the Postal Codes, now i introduce them in the toronto neighbourhood dataframe

In [9]:
import numpy as np

df_to['Latitude'] = "0"
df_to['Longitude'] = "0"

#merging dataframe that contain coordinates with the one that contains borough names
for i in df_to.index:
    for j in coord.index:
        if df_to.iloc[i, 0] == coord.iloc[j, 0]:
            df_to.iloc[i, 3] = coord.iloc[j, 1]
            df_to.iloc[i, 4] = coord.iloc[j, 2]

#checking the results            
df_to.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [10]:
df_to.shape

(103, 5)

### Step 3: Clustering Toronto Neighbourhoods

#### First: Import the libraries i will need

In [11]:
!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Find the coordinates of the center of Toronto

In [12]:
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


#### Plotting all the neighbourhoods of Toronto

In [13]:
# create map of Toronto using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_to['Latitude'], df_to['Longitude'], df_to['Borough'], df_to['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

#### Coloring the Toronto map by boroughs

In [14]:
# create map of Toronto using latitude and longitude values
map_to_bo = folium.Map(location=[latitude, longitude], zoom_start=10)

#Coloring the map by boroughs
no_borough=(int(len(df_to['Borough'].unique())))
x = np.arange(no_borough)
ys = [i + x + (i*x)**2 for i in range(no_borough)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
color=pd.DataFrame()
color['Borough']=df_to['Borough'].unique()
color['Color']=rainbow


# add markers to map
for lat, lng, borough, neighbourhood in zip(df_to['Latitude'], df_to['Longitude'], df_to['Borough'], df_to['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    colo =color[color['Borough']==borough]['Color']
    colo=colo.reset_index()
    c=colo['Color'][0]
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=0.7,
        parse_html=False).add_to(map_to_bo)
    
map_to_bo

Let's concentrate in Downtown Toronto

In [15]:
down_to = df_to[df_to['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
down_to.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.668,-79.3677
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789


In [16]:
down_to.shape

(19, 5)

In [17]:
map_downto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(down_to['Latitude'], down_to['Longitude'], down_to['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downto)  
    
map_downto

#### Now, let's explore the neirbourhoods as in the New York Exercise

In [18]:
#My credentials of Foursquare

CLIENT_ID = 'XXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXX' # your Foursquare Secret
ACCESS_TOKEN = 'NP1OSAQQOPIUCVQDN4PAHJ0CL5EUHHUZD3LMA2PFBYUJCDOE' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#### Now, let's get the top 100 venues that are in each Neighbourhood of Downtown within a radius of 500 meters.

In [19]:
#Copying the function of New York exercise

def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
downtown_venues = getNearbyVenues(names=down_to['Neighbourhood'],
                                   latitudes=down_to['Latitude'],
                                   longitudes=down_to['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


Check the shape

In [21]:
print(downtown_venues.shape)
downtown_venues.head()

(1207, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


#### Let's find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 203 uniques categories.


Analyze each Neighbourhood

In [23]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
downtown_onehot.shape

(1207, 204)

In [25]:
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.018519,0.000000,0.000000,0.00,0.00,0.018519,0.000000,0.00000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.076923,0.076923,0.076923,0.153846,0.153846,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.016393,0.000000,0.000000,0.00,0.00,0.016393,0.000000,0.00000,0.016393,0.016393
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000
4,Church and Wellesley,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.00,...,0.012658,0.012658,0.012658,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.025316
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.020000,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.00000,0.010000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,...,0.020000,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.00000,0.010000,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.010000,0.020000,0.000000,0.00,0.00,0.000000,0.010000,0.01000,0.010000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,...,0.000000,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.00000,0.010000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.065574,0.000000,0.04918,0.016393,0.000000


In [26]:
downtown_grouped.shape

(19, 204)

In [27]:
num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.06
2  Farmers Market  0.04
3      Restaurant  0.04
4     Cheese Shop  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.15
1  Airport Terminal  0.15
2   Harbor / Marina  0.08
3           Airport  0.08
4             Plane  0.08


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1      Sandwich Place  0.07
2  Italian Restaurant  0.05
3                Café  0.05
4        Burger Joint  0.03


----Christie----
                venue  freq
0       Grocery Store  0.25
1                Café  0.19
2                Park  0.12
3         Coffee Shop  0.06
4  Italian Restaurant  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.09
1     Sushi Restaurant  0.06
2  Japanese Restaurant  0.06
3           Restaurant  0.04
4  

First, let's write a function to sort the venues in descending order.



In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood:

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Seafood Restaurant,Farmers Market,Beer Bar,Restaurant,Diner,Basketball Stadium
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Plane,Rental Car Location,Sculpture Garden,Coffee Shop,Airport Lounge,Airport Food Court
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Burger Joint,Salad Place,Portuguese Restaurant,Poke Place,Middle Eastern Restaurant
3,Christie,Grocery Store,Café,Park,Italian Restaurant,Baby Store,Candy Store,Restaurant,Athletics & Sports,Nightclub,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Fast Food Restaurant


#### Cluster Neighborhoods

In [30]:
# set number of clusters
kclusters = 4

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Labels', kmeans.labels_)

downtown_merged = down_to

# merge downtown_grouped with toronto data to add latitude/longitude for each neighborhood
# I realized that I've misspelled the NeighboUrhood column name in Toronto dataframe. oops...
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775,1,Park,Playground,Trail,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.668,-79.3677,0,Pizza Place,Coffee Shop,Café,Restaurant,Bakery,Pub,Italian Restaurant,Chinese Restaurant,Butcher,Japanese Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Fast Food Restaurant
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Dessert Shop,Restaurant,Chocolate Shop
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Café,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Theater,Electronics Store


Finally, let's visualize the resulting clusters



In [32]:
#visualization libraries
import matplotlib.cm as cm
import matplotlib.colors as colors
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=9,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

Cluster 1

In [33]:
downtown_merged.loc[downtown_merged['Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Pizza Place,Coffee Shop,Café,Restaurant,Bakery,Pub,Italian Restaurant,Chinese Restaurant,Butcher,Japanese Restaurant
2,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Fast Food Restaurant
3,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Dessert Shop,Restaurant,Chocolate Shop
4,Downtown Toronto,0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Café,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Theater,Electronics Store
5,Downtown Toronto,0,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Cosmetics Shop,Restaurant,Moroccan Restaurant,Creperie,Bakery
6,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Seafood Restaurant,Farmers Market,Beer Bar,Restaurant,Diner,Basketball Stadium
7,Downtown Toronto,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Burger Joint,Salad Place,Portuguese Restaurant,Poke Place,Middle Eastern Restaurant
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gym,Bakery,Thai Restaurant,Deli / Bodega,Clothing Store,Lounge,Sushi Restaurant
9,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Scenic Lookout,Brewery,Sporting Goods Shop,Italian Restaurant,Fried Chicken Joint
10,Downtown Toronto,0,Coffee Shop,Hotel,Café,Salad Place,Restaurant,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,American Restaurant,Sushi Restaurant


Cluster 2

In [34]:
downtown_merged.loc[downtown_merged['Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


Cluster 3

In [35]:
downtown_merged.loc[downtown_merged['Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,2,Grocery Store,Café,Park,Italian Restaurant,Baby Store,Candy Store,Restaurant,Athletics & Sports,Nightclub,Coffee Shop


Cluster 4

In [36]:
downtown_merged.loc[downtown_merged['Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Plane,Rental Car Location,Sculpture Garden,Coffee Shop,Airport Lounge,Airport Food Court
